<a href="https://colab.research.google.com/github/MH-1294/ML_DL/blob/main/AI%20Model(Computer_Vision_Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
faysalmiah1721758_potato_dataset_path = kagglehub.dataset_download('faysalmiah1721758/potato-dataset')

print('Data source import complete.')


In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import kagglehub

AUTOTUNE = tf.data.AUTOTUNE

# Step 1: Download dataset
faysalmiah1721758_potato_dataset_path = kagglehub.dataset_download('faysalmiah1721758/potato-dataset')
print("Data download complete.")

DATASET_DIR = faysalmiah1721758_potato_dataset_path

# Step 2: List image paths and labels
class_names = sorted(os.listdir(DATASET_DIR))
file_paths = []
labels = []

for label_index, class_name in enumerate(class_names):
    class_dir = os.path.join(DATASET_DIR, class_name)
    for fname in os.listdir(class_dir):
        file_paths.append(os.path.join(class_dir, fname))
        labels.append(label_index)

file_paths = np.array(file_paths)
labels = np.array(labels)

# Step 3: 80/10/10 split
X_train, X_temp, y_train, y_temp = train_test_split(
    file_paths, labels, test_size=0.20, stratify=labels, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# Step 4: Dataset loading function (with optional augmentation)
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

def load_dataset(file_paths, labels, augment=False, shuffle=True):
    path_ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    def process_image(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, IMAGE_SIZE)
        image = image / 255.0  # Normalize

        if augment:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_brightness(image, max_delta=0.1)
            image = tf.image.random_contrast(image, lower=0.9, upper=1.1)

        return image, tf.one_hot(label, depth=len(class_names))

    ds = path_ds.map(process_image, num_parallel_calls=AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)

    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Step 5: Create datasets
train_ds = load_dataset(X_train, y_train, augment=True)  # Enable if you want light augmentation
val_ds = load_dataset(X_val, y_val, augment=False)
test_ds = load_dataset(X_test, y_test, augment=False)

print("Class names:", class_names)


In [ ]:
import os
import pickle
import datetime
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import numpy as np

def train_and_evaluate_model(model, model_name, train_ds, val_ds, test_ds, class_names, epochs=30):
    # Create folders
    os.makedirs('models', exist_ok=True)
    os.makedirs('history', exist_ok=True)

    # Unique model name
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    model_name = f"{model_name}_{timestamp}"
    model_path = f"models/{model_name}.h5"
    history_path = f"history/{model_name}_history.pkl"
    plot_path = f"history/{model_name}_plot.png"
    cm_path = f"history/{model_name}_confusion_matrix.png"

    # Compute class weights based on class imbalance
    print("🔍 Computing class weights...")
    y_train = []
    for _, labels in train_ds:
        y_train.extend(np.argmax(labels.numpy(), axis=1))
    class_weight = compute_class_weight(
        class_weight='balanced',
        classes=np.arange(len(class_names)),
        y=y_train
    )
    class_weight = dict(enumerate(class_weight))
    print(f"Class weights: {class_weight}")

    # Compile model
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)

    # Train
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[early_stop, checkpoint],
        class_weight=class_weight
    )

    # Save history
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)

    # Evaluate
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"\nTest Accuracy: {test_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    # Predict
    y_true, y_pred = [], []
    for images, labels in test_ds:
        preds = model.predict(images)
        y_true.extend(np.argmax(labels.numpy(), axis=1))
        y_pred.extend(np.argmax(preds, axis=1))

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(8, 8))  # increase size to fix overlap
    disp.plot(ax=ax, cmap='Blues', colorbar=False, xticks_rotation=45)
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(cm_path)
    plt.show()

    # Accuracy & Loss Plots
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig(plot_path)
    plt.show()

    print(f"\nBest model saved to: {model_path}")
    print(f"Training history saved to: {history_path}")
    print(f"Training plots saved to: {plot_path}")
    print(f"Confusion matrix saved to: {cm_path}")


In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf

def build_improved_cnn(input_shape=(256, 256, 3), num_classes=3):
    inputs = tf.keras.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 2
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 3
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 4
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 5
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 6
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Fully connected layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
# Build the lightweight CNN model
model = build_improved_cnn()

# Train and evaluate
train_and_evaluate_model(
    model=model,
    model_name="new_test_light_weight_CNN",
    train_ds=train_ds,
    val_ds=val_ds,
    test_ds=test_ds,
    class_names=class_names,
    epochs=50
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Function to make prediction on a single image
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Shape: (1, height, width, 3)

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)
    return predicted_class, confidence

# Visualize predictions on a batch of test images
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):  # Take one batch from test dataset
    for i in range(min(9, len(images))):  # Display 9 images or less if batch is smaller
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[tf.argmax(labels[i]).numpy()] if len(labels[i].shape) > 0 and labels[i].shape[-1] > 1 else class_names[labels[i].numpy()]

        plt.title(f"Actual: {actual_class}\nPredicted: {predicted_class}\nConfidence: {confidence}%")
        plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
model.save("/kaggle/working/leaf_disease_model.h5")

In [ ]:
model.save("/kaggle/working/models/lightweight_cnn_20250625-011416.h5")

In [ ]:
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        print(layer.name)


In [ ]:
Step 1

In [ ]:
# Select ONE test image (not a batch)
for image, label in test_ds.unbatch().take(3):
    input_image = image  # shape: [H, W, 3]
    input_label = label
    break


In [ ]:
# Resize the selected image
input_height, input_width = model.input_shape[1], model.input_shape[2]
image_resized = tf.image.resize(input_image, (input_height, input_width))

# Ensure float32 and correct shape
image_resized = tf.cast(image_resized, tf.float32)  # shape: [256, 256, 3]

# Predict
image_batch = tf.expand_dims(image_resized, axis=0)  # shape: [1, 256, 256, 3]
preds = model.predict(image_batch, verbose=0)[0]
pred_idx = np.argmax(preds)
confidence = round(100 * preds[pred_idx], 2)
pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()

# Grad-CAM
heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)
heatmap_resized = cv2.resize(heatmap, (input_width, input_height))

# Prepare image for masking
image_np = (image_resized.numpy() * 255).astype(np.uint8)
if image_np.shape[-1] == 1:
    image_np = np.repeat(image_np, 3, axis=-1)

# Apply leaf mask
leaf_mask = mask_leaf(image_np)
heatmap_masked = np.where(leaf_mask == 255, heatmap_resized, 0)

# Show visualization
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(image_np)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(heatmap_resized, cmap='plasma')
plt.title("Grad-CAM (Full)")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(heatmap_masked, cmap='plasma')
plt.title("Grad-CAM (Masked to Leaf)")
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
print("Grad-CAM max:", np.max(heatmap_resized))
print("Masked Grad-CAM max:", np.max(heatmap_masked))


In [ ]:
print("Non-zero pixels (before masking):", np.count_nonzero(heatmap_resized))
print("Non-zero pixels (after masking):", np.count_nonzero(heatmap_masked))


K-means

In [ ]:
from sklearn.cluster import KMeans

def apply_three_way_kmeans(heatmap, n_clusters=3):
    flat = heatmap.flatten().reshape(-1, 1)

    # Avoid clustering error if image is flat
    if np.std(flat) < 1e-5:
        return np.zeros_like(heatmap, dtype=np.uint8)

    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto').fit(flat)
    clustered = kmeans.labels_.reshape(heatmap.shape)

    # Sort clusters by intensity
    means = [np.mean(flat[kmeans.labels_ == i]) for i in range(n_clusters)]
    sorted_indices = np.argsort(means)

    label_map = np.zeros_like(clustered)
    for new_label, original_label in enumerate(sorted_indices):
        label_map[clustered == original_label] = new_label

    return label_map.astype(np.uint8)  # 0 = background, 1 = uncertain, 2 = strong attention


In [ ]:
def overlay_clustered_attention(image_np, cluster_mask):
    """
    Overlay 3 attention zones on top of original image.
    """
    output = image_np.copy()

    # Define RGB colors for each zone
    colors = {
        0: [180, 180, 180],  # gray = ignored
        1: [255, 165, 0],    # orange = uncertain
        2: [255, 0, 0]       # red = strong attention
    }

    for region, color in colors.items():
        mask = (cluster_mask == region)
        for c in range(3):
            output[:, :, c] = np.where(
                mask,
                0.5 * output[:, :, c] + 0.5 * color[c],
                output[:, :, c]
            )

    return output.astype(np.uint8)


In [ ]:
# Apply 3-way KMeans clustering to masked heatmap
cluster_mask = apply_three_way_kmeans(heatmap_masked)
cluster_mask = cv2.resize(cluster_mask.astype(np.uint8), (input_width, input_height), interpolation=cv2.INTER_NEAREST)

# Overlay clustered attention zones
overlayed_image = overlay_clustered_attention(image_np, cluster_mask)

# Visualize final result
plt.figure(figsize=(6, 6))
plt.imshow(overlayed_image)
plt.title(f"3-Way Clustered Attention\n(Predicted: {pred_label} | {confidence}%)")
plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
from sklearn.cluster import KMeans

# Function: Generate Grad-CAM
def generate_gradcam(model, image, class_index, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(tf.expand_dims(image, axis=0))
        loss = predictions[:, class_index]
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Function: Leaf Mask (HSV green filter)
def mask_leaf(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_green = np.array([25, 40, 40])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask

# Function: 3-Way KMeans
def apply_three_way_kmeans(heatmap, n_clusters=3):
    flat = heatmap.flatten().reshape(-1, 1)
    if np.std(flat) < 1e-5:
        return np.zeros_like(heatmap, dtype=np.uint8)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto').fit(flat)
    clustered = kmeans.labels_.reshape(heatmap.shape)
    means = [np.mean(flat[kmeans.labels_ == i]) for i in range(n_clusters)]
    sorted_indices = np.argsort(means)
    label_map = np.zeros_like(clustered)
    for new_label, original_label in enumerate(sorted_indices):
        label_map[clustered == original_label] = new_label
    return label_map.astype(np.uint8)

# Function: Overlay Clustered Attention
def overlay_clustered_attention(image_np, cluster_mask):
    output = image_np.copy()
    colors = {
        0: [180, 180, 180],  # background (gray)
        1: [255, 165, 0],    # uncertain (orange)
        2: [255, 0, 0]       # lesion (red)
    }
    for region, color in colors.items():
        mask = (cluster_mask == region)
        for c in range(3):
            output[:, :, c] = np.where(
                mask,
                0.5 * output[:, :, c] + 0.5 * color[c],
                output[:, :, c]
            )
    return output.astype(np.uint8)

# MAIN: Visualize 5 Test Images Side-by-Side
plt.figure(figsize=(10, 10))
input_height, input_width = model.input_shape[1], model.input_shape[2]
count = 0

for image, label in test_ds.unbatch().take(5):
    image_resized = tf.image.resize(image, (input_height, input_width))
    image_resized = tf.cast(image_resized, tf.float32)
    image_batch = tf.expand_dims(image_resized, axis=0)
    preds = model.predict(image_batch, verbose=0)[0]
    pred_idx = np.argmax(preds)
    confidence = round(100 * preds[pred_idx], 2)
    pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()

    # Grad-CAM and masking
    heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)
    heatmap_resized = cv2.resize(heatmap, (input_width, input_height))
    image_np = (image_resized.numpy() * 255).astype(np.uint8)
    if image_np.shape[-1] == 1:
        image_np = np.repeat(image_np, 3, axis=-1)
    leaf_mask = mask_leaf(image_np)
    heatmap_masked = np.where(leaf_mask == 255, heatmap_resized, 0)
    cluster_mask = apply_three_way_kmeans(heatmap_masked)
    cluster_mask = cv2.resize(cluster_mask.astype(np.uint8), (input_width, input_height), interpolation=cv2.INTER_NEAREST)
    overlayed_image = overlay_clustered_attention(image_np, cluster_mask)

    # Plot original + clustered Grad-CAM side-by-side
    plt.subplot(5, 2, 2 * count + 1)
    plt.imshow(image_np)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(5, 2, 2 * count + 2)
    plt.imshow(overlayed_image)
    plt.title(f"Clustered Attention\n{pred_label} ({confidence}%)")
    plt.axis("off")

    count += 1

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 20))
input_height, input_width = model.input_shape[1], model.input_shape[2]
count = 0

for image, label in test_ds.unbatch().take(10):
    image_resized = tf.image.resize(image, (input_height, input_width))
    image_resized = tf.cast(image_resized, tf.float32)
    image_batch = tf.expand_dims(image_resized, axis=0)
    preds = model.predict(image_batch, verbose=0)[0]
    pred_idx = np.argmax(preds)
    confidence = round(100 * preds[pred_idx], 2)
    pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()

    # Grad-CAM + masking
    heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)
    heatmap_resized = cv2.resize(heatmap, (input_width, input_height))
    image_np = (image_resized.numpy() * 255).astype(np.uint8)
    if image_np.shape[-1] == 1:
        image_np = np.repeat(image_np, 3, axis=-1)
    leaf_mask = mask_leaf(image_np)
    heatmap_masked = np.where(leaf_mask == 255, heatmap_resized, 0)
    cluster_mask = apply_three_way_kmeans(heatmap_masked)
    cluster_mask = cv2.resize(cluster_mask.astype(np.uint8), (input_width, input_height), interpolation=cv2.INTER_NEAREST)
    overlayed_image = overlay_clustered_attention(image_np, cluster_mask)

    # Plot side-by-side
    plt.subplot(10, 2, 2 * count + 1)
    plt.imshow(image_np)
    plt.title("Original")
    plt.axis("off")

    plt.subplot(10, 2, 2 * count + 2)
    plt.imshow(overlayed_image)
    plt.title(f"Overlay - {pred_label} ({confidence}%)")
    plt.axis("off")

    count += 1

plt.tight_layout()
plt.show()


In [ ]:
lesion_pixels = np.sum(cluster_mask == 2)
uncertain_pixels = np.sum(cluster_mask == 1)
background_pixels = np.sum(cluster_mask == 0)
leaf_pixels = np.sum(leaf_mask == 255)

lesion_ratio = round((lesion_pixels / leaf_pixels) * 100, 2)
print(f"[{count}] Lesion focus: {lesion_ratio}% of leaf area")


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
from sklearn.cluster import KMeans

# Prepare containers
good_samples = []
bad_samples = []

# Model input
input_height, input_width = model.input_shape[1], model.input_shape[2]

# Loop through test set
for image, label in test_ds.unbatch().take(100):  # Scan up to 100
    image_resized = tf.image.resize(image, (input_height, input_width))
    image_resized = tf.cast(image_resized, tf.float32)
    image_batch = tf.expand_dims(image_resized, axis=0)
    preds = model.predict(image_batch, verbose=0)[0]
    pred_idx = np.argmax(preds)
    confidence = round(100 * preds[pred_idx], 2)
    pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()

    # Grad-CAM
    heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)
    heatmap_resized = cv2.resize(heatmap, (input_width, input_height))
    image_np = (image_resized.numpy() * 255).astype(np.uint8)
    if image_np.shape[-1] == 1:
        image_np = np.repeat(image_np, 3, axis=-1)

    # Mask + Clustering
    leaf_mask = mask_leaf(image_np)
    heatmap_masked = np.where(leaf_mask == 255, heatmap_resized, 0)
    cluster_mask = apply_three_way_kmeans(heatmap_masked)
    cluster_mask = cv2.resize(cluster_mask.astype(np.uint8), (input_width, input_height), interpolation=cv2.INTER_NEAREST)
    overlayed_image = overlay_clustered_attention(image_np, cluster_mask)

    # Calculate lesion attention ratio
    lesion_pixels = np.sum(cluster_mask == 2)
    leaf_pixels = np.sum(leaf_mask == 255)
    if leaf_pixels == 0:
        continue
    lesion_ratio = (lesion_pixels / leaf_pixels) * 100

    # Decide if it's good or bad
    if 10 <= lesion_ratio <= 45:
        good_samples.append((image_np, overlayed_image, pred_label, confidence, lesion_ratio))
    elif lesion_ratio < 5 or lesion_ratio > 70:
        bad_samples.append((image_np, overlayed_image, pred_label, confidence, lesion_ratio))

    # Stop if we have enough
    if len(good_samples) >= 3 and len(bad_samples) >= 3:
        break

# Plot figure
plt.figure(figsize=(12, 12))
for i in range(3):
    # Good sample row
    img_orig, img_overlay, label, conf, ratio = good_samples[i]
    plt.subplot(3, 4, 4*i + 1)
    plt.imshow(img_orig)
    plt.title(f"Good-{i+1}: Original")
    plt.axis("off")

    plt.subplot(3, 4, 4*i + 2)
    plt.imshow(img_overlay)
    plt.title(f"{label} ({conf}%)\nLesion: {round(ratio, 1)}%")
    plt.axis("off")

    # Bad sample row
    img_orig, img_overlay, label, conf, ratio = bad_samples[i]
    plt.subplot(3, 4, 4*i + 3)
    plt.imshow(img_orig)
    plt.title(f"Bad-{i+1}: Original")
    plt.axis("off")

    plt.subplot(3, 4, 4*i + 4)
    plt.imshow(img_overlay)
    plt.title(f"{label} ({conf}%)\nLesion: {round(ratio, 1)}%")
    plt.axis("off")

plt.tight_layout()
plt.suptitle("Good vs Bad Grad-CAM Interpretability (3-Way Clustering)", fontsize=16, y=1.02)
plt.show()


In [ ]:
plt.savefig("/kaggle/working/good_vs_bad_gradcam.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Predict the class and get confidence
image_resized = tf.image.resize(input_image, (input_height, input_width))
image_resized = tf.convert_to_tensor(image_resized, dtype=tf.float32)
image_resized = tf.ensure_shape(image_resized, [input_height, input_width, 3])  # Ensure shape

# Expand dimensions for batch input
image_batch = tf.expand_dims(image_resized, axis=0)

# Run model prediction
preds = model.predict(image_batch, verbose=0)[0]
pred_idx = np.argmax(preds)
confidence = round(100 * preds[pred_idx], 2)
pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()

# Grad-CAM generation
heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)
heatmap_resized = cv2.resize(heatmap, (input_width, input_height))

# Convert to NumPy for OpenCV
image_np = (image_resized.numpy() * 255).astype(np.uint8)
if image_np.shape[-1] == 1:
    image_np = np.repeat(image_np, 3, axis=-1)

# Generate the leaf mask
leaf_mask = mask_leaf(image_np)

# Apply the mask to Grad-CAM
heatmap_masked = np.where(leaf_mask == 255, heatmap_resized, 0)

# Visualize results
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(image_np)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(heatmap_resized, cmap='plasma')
plt.title("Grad-CAM (Full)")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(heatmap_masked, cmap='plasma')
plt.title("Grad-CAM (Masked)")
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import cv2
import numpy as np

def mask_leaf(image):
    """
    Takes a NumPy RGB image (0–255) and returns a binary mask of the leaf area.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Define green color range in HSV (tune if needed)
    lower_green = np.array([25, 40, 40])
    upper_green = np.array([85, 255, 255])

    # Create a mask for green regions
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Optional: clean noise using morphological operations
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    return mask  # dtype=uint8, shape=(H, W), values in {0, 255}


In [ ]:
# 1. Predict class and confidence
preds = model.predict(tf.expand_dims(image_resized, axis=0), verbose=0)[0]
pred_idx = np.argmax(preds)
confidence = round(100 * preds[pred_idx], 2)
pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()

# 2. Generate Grad-CAM
heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)

# 3. Resize heatmap to match image dimensions
heatmap_resized = cv2.resize(heatmap, (input_width, input_height))

# 4. Convert image to NumPy for OpenCV
image_np = (image_resized.numpy() * 255).astype(np.uint8)
if image_np.shape[-1] == 1:
    image_np = np.repeat(image_np, 3, axis=-1)

# 5. Generate leaf-only mask using HSV color filtering
leaf_mask = mask_leaf(image_np)  # Output shape: (H, W), values: 0 or 255

# 6. Apply the mask to the Grad-CAM heatmap
heatmap_masked = np.where(leaf_mask == 255, heatmap_resized, 0)


In [ ]:
# Generate Grad-CAM
heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)

# Resize heatmap to original image size
heatmap_resized = cv2.resize(heatmap, (input_width, input_height))

# Convert image to uint8 for OpenCV
image_np = (image_resized.numpy() * 255).astype(np.uint8)

# Generate mask for leaf only
leaf_mask = mask_leaf(image_np)

# Apply the mask: keep only values on the leaf
heatmap_masked = np.where(leaf_mask == 255, heatmap_resized, 0)


Three way k means

In [ ]:
def apply_three_way_kmeans(heatmap, n_clusters=3):
    flat = heatmap.flatten().reshape(-1, 1)

    # Early exit: if all values are nearly the same
    if np.std(flat) < 1e-5:
        return np.zeros_like(heatmap, dtype=np.uint8)  # fallback: all background

    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto').fit(flat)
    clustered = kmeans.labels_.reshape(heatmap.shape)

    # Sort cluster labels by average intensity
    means = [np.mean(flat[kmeans.labels_ == i]) for i in range(n_clusters)]
    sorted_indices = np.argsort(means)

    label_map = np.zeros_like(clustered)
    for new_label, old_label in enumerate(sorted_indices):
        label_map[clustered == old_label] = new_label

    return label_map.astype(np.uint8)  # 0=background, 1=uncertain, 2=lesion


In [ ]:
def overlay_clustered_attention(image, cluster_mask):
    image_np = (image.numpy() * 255).astype(np.uint8)
    if image_np.shape[-1] == 1:
        image_np = np.repeat(image_np, 3, axis=-1)

    # Define color mapping: background (gray), uncertain (orange), strong (red)
    colors = {
        0: [180, 180, 180],   # background
        1: [255, 165, 0],     # uncertain
        2: [255, 0, 0]        # strong attention
    }

    overlay = image_np.copy()
    for region, color in colors.items():
        mask = (cluster_mask == region)
        for c in range(3):
            overlay[:, :, c] = np.where(
                mask,
                0.5 * overlay[:, :, c] + 0.5 * color[c],
                overlay[:, :, c]
            )

    return overlay.astype(np.uint8)


In [ ]:
def predict_and_visualize_overlay(model, test_ds, class_names):
    target_labels = {"Healthy", "Early Blight", "Late Blight"}
    selected = {}
    for image, label in test_ds.unbatch():
        label_idx = label.numpy() if isinstance(label.numpy(), np.integer) else np.argmax(label.numpy())
        class_name = class_names[label_idx].split("___")[-1].replace("_", " ").title()
        if class_name in target_labels and class_name not in selected:
            selected[class_name] = (image, label_idx)
        if len(selected) == len(target_labels):
            break
    if len(selected) < 3:
        raise ValueError(f"Missing classes: {set(target_labels) - set(selected)}")

    plt.figure(figsize=(10, 12))
    for i, class_name in enumerate(["Healthy", "Early Blight", "Late Blight"]):
        image, true_idx = selected[class_name]
        image_resized = tf.image.resize(image, (input_height, input_width))
        preds = model.predict(tf.expand_dims(image_resized, axis=0), verbose=0)[0]
        pred_idx = np.argmax(preds)
        pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()
        confidence = round(100 * preds[pred_idx], 2)

        # Generate Grad-CAM
        heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)

        # Cluster the heatmap
        cluster_mask = apply_three_way_kmeans(heatmap)

        cluster_mask = cv2.resize(cluster_mask.astype(np.uint8), (input_width, input_height), interpolation=cv2.INTER_NEAREST)


        # Overlay clustered mask
        overlayed_cluster = overlay_clustered_attention(image_resized, cluster_mask)

        # Plot original image
        ax1 = plt.subplot(3, 2, 2 * i + 1)
        ax1.imshow(image_resized.numpy().astype("float32"))
        ax1.axis("off")
        ax1.set_title(f"True: {class_name}", fontsize=10)

        # Plot clustered attention
        ax2 = plt.subplot(3, 2, 2 * i + 2)
        ax2.imshow(overlayed_cluster)
        ax2.axis("off")
        ax2.set_title(f"Predicted: {pred_label} ({confidence}%)", fontsize=9)

    plt.tight_layout()
    plt.show()


In [ ]:
predict_and_visualize_overlay(model, test_ds, class_names)

Grad CAM#

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load model
model = tf.keras.models.load_model("/kaggle/working/models/lightweight_cnn_20250625-011416.h5")
model.trainable = False  # Ensure BatchNorm and Dropout act in inference mode
input_height, input_width = model.input_shape[1], model.input_shape[2]

# Mid-level convolutional layer
last_conv_layer = "conv2d_4"  # Double-check this with model.summary()

# Generate Grad-CAM heatmap
def generate_gradcam(model, image, class_index, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(tf.expand_dims(image, axis=0))
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)[0]
    if grads is None:
        raise ValueError("Gradients are None. Ensure model has trainable layers and correct conv layer.")

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Generate overlay image
def generate_gradcam_overlay(image, heatmap, alpha=0.5, colormap=cv2.COLORMAP_PLASMA, threshold=0.4):
    # Convert heatmap to uint8 and apply color map
    heatmap_uint8 = np.uint8(255 * heatmap)
    color_heatmap = cv2.applyColorMap(heatmap_uint8, colormap)
    color_heatmap = cv2.cvtColor(color_heatmap, cv2.COLOR_BGR2RGB)

    # Convert image to uint8
    image_np = (image.numpy() * 255).astype(np.uint8)
    if image_np.shape[-1] == 1:
        image_np = np.repeat(image_np, 3, axis=-1)

    # Resize heatmap to match image size
    color_heatmap = cv2.resize(color_heatmap, (image_np.shape[1], image_np.shape[0]))
    heatmap_resized = cv2.resize(heatmap, (image_np.shape[1], image_np.shape[0]))  # for mask

    # Apply threshold-based blending
    mask = heatmap_resized > threshold
    blended = cv2.addWeighted(image_np, 1 - alpha, color_heatmap, alpha, 0)
    overlayed = image_np.copy()
    overlayed[mask] = blended[mask]

    return overlayed


# Visualize Grad-CAM for three target classes
def predict_and_visualize_overlay(model, test_ds, class_names):
    target_labels = {"Healthy", "Early Blight", "Late Blight"}
    selected = {}
    for image, label in test_ds.unbatch():
        label_idx = label.numpy() if isinstance(label.numpy(), np.integer) else np.argmax(label.numpy())
        class_name = class_names[label_idx].split("___")[-1].replace("_", " ").title()
        if class_name in target_labels and class_name not in selected:
            selected[class_name] = (image, label_idx)
        if len(selected) == len(target_labels):
            break
    if len(selected) < 3:
        raise ValueError(f"Missing classes: {set(target_labels) - set(selected)}")

    plt.figure(figsize=(10, 12))
    for i, class_name in enumerate(["Healthy", "Early Blight", "Late Blight"]):
        image, true_idx = selected[class_name]
        image_resized = tf.image.resize(image, (input_height, input_width))
        preds = model.predict(tf.expand_dims(image_resized, axis=0), verbose=0)[0]
        pred_idx = np.argmax(preds)
        pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()
        confidence = round(100 * preds[pred_idx], 2)

        # Generate Grad-CAM
        heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)

        # Overlay
        overlayed_image = generate_gradcam_overlay(image_resized, heatmap, alpha=0.5, threshold=0.4)

        # Plot original
        ax1 = plt.subplot(3, 2, 2 * i + 1)
        ax1.imshow(image_resized.numpy().astype("float32"))
        ax1.axis("off")
        ax1.set_title(f"True: {class_name}", fontsize=10)

        # Plot Grad-CAM overlay
        ax2 = plt.subplot(3, 2, 2 * i + 2)
        im = ax2.imshow(overlayed_image)
        ax2.axis("off")
        ax2.set_title(f"Predicted: {pred_label} ({confidence}%)", fontsize=9)
        cbar = plt.colorbar(
            plt.cm.ScalarMappable(cmap="plasma", norm=plt.Normalize(vmin=0, vmax=1)),
            ax=ax2, fraction=0.046, pad=0.04
        )
        cbar.set_label("Model Attention", fontsize=8)

    plt.tight_layout()
    plt.show()


In [ ]:
predict_and_visualize_overlay(model, test_ds, class_names)

In [ ]:
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, (input_height, input_width)), y))


In [ ]:
predict_and_visualize_overlay(model, test_ds, class_names)

In [ ]:
from sklearn.cluster import KMeans

def apply_three_way_kmeans(heatmap, n_clusters=3):
    flat = heatmap.flatten().reshape(-1, 1)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(flat)
    clustered = kmeans.labels_.reshape(heatmap.shape)

    # Sort labels by mean value (e.g., 0 = low, 2 = high)
    means = [np.mean(flat[kmeans.labels_ == i]) for i in range(n_clusters)]
    sorted_idx = np.argsort(means)

    label_map = np.zeros_like(clustered)
    for new_label, old_label in enumerate(sorted_idx):
        label_map[clustered == old_label] = new_label

    return label_map  # 0 = negative, 1 = boundary, 2 = positive


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2

# Load model
model = tf.keras.models.load_model("/kaggle/working/models/lightweight_cnn_20250625-011416.h5")
input_height, input_width = model.input_shape[1], model.input_shape[2]
last_conv_layer = "conv2d_2"

# Grad-CAM heatmap generator
def generate_gradcam(model, image, class_index, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(tf.expand_dims(image, axis=0))
        loss = predictions[:, class_index]
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Overlay only attention area
def generate_gradcam_overlay(image, heatmap, alpha=0.5, colormap=cv2.COLORMAP_PLASMA, threshold=0.4):
    heatmap_uint8 = np.uint8(255 * heatmap)
    color_heatmap = cv2.applyColorMap(heatmap_uint8, colormap)
    color_heatmap = cv2.cvtColor(color_heatmap, cv2.COLOR_BGR2RGB)
    image_np = (image.numpy() * 255).astype(np.uint8)
    if image_np.shape[-1] == 1:
        image_np = np.repeat(image_np, 3, axis=-1)
    mask = heatmap > threshold
    overlayed = image_np.copy()
    for c in range(3):
        overlayed[:, :, c] = np.where(
            mask,
            cv2.addWeighted(image_np[:, :, c], 1 - alpha, color_heatmap[:, :, c], alpha, 0),
            image_np[:, :, c]
        )
    return overlayed

# Aligned horizontal layout with uniform spacing
def predict_and_visualize_aligned_grid(model, test_ds, class_names):
    target_labels = {"Healthy", "Early Blight", "Late Blight"}
    selected = {}
    for image, label in test_ds.unbatch():
        label_idx = label.numpy() if isinstance(label.numpy(), np.integer) else np.argmax(label.numpy())
        class_name = class_names[label_idx].split("___")[-1].replace("_", " ").title()
        if class_name in target_labels and class_name not in selected:
            selected[class_name] = (image, label_idx)
        if len(selected) == len(target_labels):
            break
    if len(selected) < 3:
        raise ValueError(f"Missing: {set(target_labels) - set(selected)}")

    fig = plt.figure(figsize=(15, 6))
    gs = gridspec.GridSpec(2, 3, figure=fig, hspace=0.15, wspace=0.1)

    for col, class_name in enumerate(["Early Blight", "Late Blight", "Healthy"]):
        image, true_idx = selected[class_name]
        image_resized = tf.image.resize(image, (input_height, input_width))
        preds = model.predict(tf.expand_dims(image_resized, axis=0), verbose=0)[0]
        pred_idx = np.argmax(preds)
        pred_label = class_names[pred_idx].split("___")[-1].replace("_", " ").title()
        confidence = round(100 * preds[pred_idx], 2)

        heatmap = generate_gradcam(model, image_resized, pred_idx, last_conv_layer)
        heatmap_resized = cv2.resize(heatmap, (input_width, input_height))
        overlayed = generate_gradcam_overlay(image_resized, heatmap_resized)

        # Row 0: Original Image
        ax1 = fig.add_subplot(gs[0, col])
        ax1.imshow(image_resized.numpy().astype("float32"))
        ax1.axis("off")
        ax1.set_title("Original Image", fontsize=10)

        # Row 1: Grad-CAM Overlay
        ax2 = fig.add_subplot(gs[1, col])
        im = ax2.imshow(overlayed)
        ax2.axis("off")
        ax2.set_title(f"{class_name} Class\nPredicted: {pred_label} ({confidence}%)", fontsize=9)

        # Add colorbar (aligned right)
        cbar = fig.colorbar(
            plt.cm.ScalarMappable(cmap="plasma", norm=plt.Normalize(vmin=0, vmax=1)),
            ax=ax2, fraction=0.046, pad=0.03
        )
        cbar.set_label("Model Attention", fontsize=8)

    plt.tight_layout()
    plt.show()


In [ ]:
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, (input_height, input_width)), y))
predict_and_visualize_aligned_grid(model, test_ds, class_names)


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from tensorflow.keras.models import Model

def generate_gradcam_heatmap(model, img_array, last_conv_layer_name):
    grad_model = Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def create_binary_mask(heatmap, threshold=0.5):
    binary_mask = (heatmap > threshold).astype(np.uint8)
    return binary_mask

def mask_background(image, binary_mask):
    masked_image = image * np.expand_dims(binary_mask, axis=-1)
    return masked_image.astype(np.uint8)

def predict_and_visualize_gradcam_lime(model, image, class_names, last_conv_layer_name):
    # Step 1: Generate Grad-CAM heatmap
    img_array = tf.expand_dims(image, 0) / 255.0
    heatmap = generate_gradcam_heatmap(model, img_array, last_conv_layer_name)

    # Step 2: Create binary attention mask
    binary_mask = create_binary_mask(heatmap)

    # Step 3: Mask out the background
    masked_image = mask_background((image * 255).astype(np.uint8), binary_mask)

    # Step 4: Apply LIME on the masked image
    explainer = lime_image.LimeImageExplainer(random_state=42)

    def lime_predict_fn(images):
        images_tf = tf.convert_to_tensor(images, dtype=tf.float32) / 255.0
        return model.predict(images_tf)

    explanation = explainer.explain_instance(
        masked_image.astype('double'),
        classifier_fn=lime_predict_fn,
        top_labels=1,
        hide_color=0,
        num_samples=1000
    )

    temp, mask = explanation.get_image_and_mask(
        label=0,
        positive_only=False,
        num_features=10,
        hide_rest=False
    )

    lime_overlay = mark_boundaries(masked_image, mask, color=(1, 0, 0), mode='thick')

    plt.figure(figsize=(10, 5))
    plt.style.use('dark_background')

    plt.subplot(1, 2, 1)
    plt.imshow(masked_image)
    plt.axis('off')
    plt.title("Masked Image", color='white')

    plt.subplot(1, 2, 2)
    plt.imshow(lime_overlay)
    plt.axis('off')
    plt.title("LIME Overlay", color='white')

    plt.tight_layout()
    plt.savefig('gradcam_lime.png')
    plt.close()

# Example usage (assuming model, image, class_names, and last_conv_layer_name are defined)
# predict_and_visualize_gradcam_lime(model, image, class_names, 'last_conv_layer')

In [ ]:
predict_and_visualize_lime_with_bg_removed(model, test_ds, class_names)


In [ ]:
model.save("/kaggle/working/Custom_CNN.keras")